In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flair==0.11.3 &> /dev/null

In [ ]:
from glob import glob
dataset = glob("/content/drive/MyDrive/gold-random-json/*")

## Data Preprocessing

In [ ]:
import json
from itertools import product
from flair.data import Sentence
from flair.tokenization import TokenizerWrapper
from nltk import wordpunct_tokenize

train_corpus = []
dev_corpus = []

for folder,corpus_type in product(dataset,["train","dev"]):
    for line in open(folder + '/' + corpus_type + '.jsonl'):
        obj = json.loads(line)
        sentence = Sentence(obj["text"],use_tokenizer = TokenizerWrapper(wordpunct_tokenize))

        for span_labels in obj["location_mentions"]:
            s,e = span_labels["start_offset"],span_labels["end_offset"]
            s_tok,e_tok = -1,-1
            for i,x in zip(range(len(sentence)),sentence):
                if s <= x.start_pos and x.end_pos <= e :
                    s_tok = s_tok if s_tok != -1 else i
                    e_tok = i
            try:
                sentence[s_tok:e_tok + 1].add_label("ner",span_labels["type"])  
            except:
                pass
        globals()[corpus_type + '_corpus'].append(sentence)
    
    

## Training

In [ ]:
from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings,FlairEmbeddings,StackedEmbeddings,TransformerWordEmbeddings
from flair.trainers import ModelTrainer
from flair.data import Corpus

corpus = Corpus(train_corpus,dev_corpus) # store train and test data
corpus.downsample(0.35) #downsample to 35% 

### GLOVE + BiLSTM + CRF

In [ ]:
label_type = 'ner' #Name-Entity Recognition (NER): It can recognise whether a 
                    #word represents a person, location or names in the text.


label_dict = corpus.make_label_dictionary(label_type=label_type) #Create a label dictionary from ner and data
print(label_dict)

embedding_types = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]
#The three embedding models will be concatenated and should give state of the art results.
#Document Embeddings generate one embedding for an entire text. 
#The produced embeddings are PyTorch vectors. 
#There are two different methods using the word embeddings to obtain a document embedding

embeddings = StackedEmbeddings(embeddings=embedding_types)

tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

trainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.08,
              mini_batch_size=32,
              max_epochs=16)

2023-04-08 23:32:22,977 Computing label dictionary. Progress:


4534it [00:00, 48000.84it/s]

2023-04-08 23:32:23,118 Dictionary created for label 'ner' with 13 values: Country (seen 1594 times), State (seen 1398 times), City/town (seen 1219 times), Island (seen 307 times), Human-made Point-of-Interest (seen 78 times), County (seen 70 times), Natural Point-of-Interest (seen 53 times), District (seen 49 times), Continent (seen 36 times), Neighborhood (seen 30 times), Road/street (seen 19 times), Other locations (seen 14 times)
Dictionary with 13 tags: <unk>, Country, State, City/town, Island, Human-made Point-of-Interest, County, Natural Point-of-Interest, District, Continent, Neighborhood, Road/street, Other locations


2023-04-08 23:32:23,769 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpk43ba7h8


100%|██████████| 160000128/160000128 [00:07<00:00, 20487359.02B/s]

...

2023-04-08 23:41:31,170 Evaluating as a multi-label problem: False
2023-04-08 23:41:31,188 DEV : loss 0.049965061247348785 - f1-score (micro avg)  0.8421


2023-04-08 23:41:31,273 BAD EPOCHS (no improvement): 3
2023-04-08 23:41:33,057 ----------------------------------------------------------------------------------------------------
2023-04-08 23:41:33,062 loading file resources/taggers/sota-ner-flair/best-model.pt
2023-04-08 23:41:34,412 SequenceTagger predicts: Dictionary with 51 tags: O, S-Country, B-Country, E-Country, I-Country, S-State, B-State, E-State, I-State, S-City/town, B-City/town, E-City/town, I-City/town, S-Island, B-Island, E-Island, I-Island, S-Human-made Point-of-Interest, B-Human-made Point-of-Interest, E-Human-made Point-of-Interest, I-Human-made Point-of-Interest, S-County, B-County, E-County, I-County, S-Natural Point-of-Interest, B-Natural Point-of-Interest, E-Natural Point-of-Interest, I-Natural Point-of-Interest, S-District, B-District, E-District, I-District, S-Continent, B-Continent, E-Continent, I-Continent, S-Neighborhood, B-Neighborhood, E-Neighborhood, I-Neighborhood, S-Road/street, B-Road/street, E-Road/st

100%|██████████| 16/16 [00:06<00:00,  2.42it/s]

2023-04-08 23:41:41,347 Evaluating as a multi-label problem: False
2023-04-08 23:41:41,372 0.8195	0.7837	0.8012	0.7054
2023-04-08 23:41:41,373 
Results:
- F-score (micro) 0.8012
- F-score (macro) 0.3995
- Accuracy 0.7054

By class:
                              precision    recall  f1-score   support

                     Country     0.9036    0.9146    0.9091       164
                       State     0.9216    0.8598    0.8896       164
                   City/town     0.5827    0.7629    0.6607        97
                      Island     0.8077    0.7241    0.7636        29
                      County     0.8571    0.4286    0.5714        14
   Natural Point-of-Interest     1.0000    0.2000    0.3333        10
Human-made Point-of-Interest     0.0000    0.0000    0.0000        10
                    District     0.0000    0.0000    0.0000         8
                 Road/street     0.0000    0.0000    0.0000         4
                   Continent     1.0000    0.5000    0.6667        

{'test_score': 0.8012170385395536,
 'dev_score_history': [0.5864156018829859,
  0.7629733520336606,
  0.7909215955983495,
  0.7969507969507968,
  0.8103683492496591,
  0.8338804990151019,
  0.8321263989466755,
  0.8433420365535248,
  0.8459016393442623,
  0.8387942332896461,
  0.8297455968688845,
  0.8272727272727273,
  0.8407310704960836,
  0.8390501319261214,
  0.8407894736842105,
  0.8421052631578947],
 'train_loss_history': [0.29591518258124644,
  0.12780638969223454,
  0.10420612382003334,
  0.09320439881275348,
  0.08493016419369469,
  0.08005953663002992,
  0.07538931812670736,
  0.07131873515221727,
  0.06775897024080119,
  0.06528490034178705,
  0.06233132578392103,
  0.06019205521856355,
  0.05859153641687127,
  0.05340869543243447,
  0.05241081952297504,
  0.05170304166125478],
 'dev_loss_history': [0.13423362374305725,
  0.07759781181812286,
  0.06737015396356583,
  0.06296397000551224,
  0.05976879969239235,
  0.055345818400382996,
  0.05467117205262184,
  0.05256162583827

In [ ]:
# del tagger,trainer,embeddings

### Transformer

In [ ]:
# label_type = 'ner'

# label_dict = corpus.make_label_dictionary(label_type=label_type)
# print(label_dict)

# embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
#                                        layers="-1",
#                                        subtoken_pooling="first",
#                                        fine_tune=True,
#                                        use_context=True,
#                                        )

# tagger = SequenceTagger(hidden_size=256,
#                         embeddings=embeddings,
#                         tag_dictionary=label_dict,
#                         tag_type='ner',
#                         use_crf=False,
#                         use_rnn=False,
#                         reproject_embeddings=False,
#                         )

# trainer = ModelTrainer(tagger, corpus)

# trainer.fine_tune('resources/taggers/sota-ner-flair',
#               learning_rate=0.00085,
#               mini_batch_size=16,
#               max_epochs=8)

## INFERENCE

In [ ]:
model = SequenceTagger.load('./resources/taggers/sota-ner-flair/final-model.pt') #loading model

2023-04-08 23:41:41,407 loading file ./resources/taggers/sota-ner-flair/final-model.pt
2023-04-08 23:41:43,082 SequenceTagger predicts: Dictionary with 51 tags: O, S-Country, B-Country, E-Country, I-Country, S-State, B-State, E-State, I-State, S-City/town, B-City/town, E-City/town, I-City/town, S-Island, B-Island, E-Island, I-Island, S-Human-made Point-of-Interest, B-Human-made Point-of-Interest, E-Human-made Point-of-Interest, I-Human-made Point-of-Interest, S-County, B-County, E-County, I-County, S-Natural Point-of-Interest, B-Natural Point-of-Interest, E-Natural Point-of-Interest, I-Natural Point-of-Interest, S-District, B-District, E-District, I-District, S-Continent, B-Continent, E-Continent, I-Continent, S-Neighborhood, B-Neighborhood, E-Neighborhood, I-Neighborhood, S-Road/street, B-Road/street, E-Road/street, I-Road/street, S-Other locations, B-Other locations, E-Other locations, I-Other locations, <START>


In [ ]:
folder_to_check = '/content/drive/MyDrive/gold-random-json/kerala_floods_2018/' # END PATH WITH / symbol 

import os

for file in glob(folder_to_check + "*"):
    if "train" not in file:
        continue
    for line in open(file,'r'):
        obj = json.loads(line)
        s = Sentence(obj["text"],use_tokenizer = TokenizerWrapper(wordpunct_tokenize))
        model.predict(s) #model is being used to predict on sentence s
        outp =  {}
        outp["tweet_id"] = obj["tweet_id"]
        outp["location_mentions"] = []
        for e in s.labels:
            outp["location_mentions"].append({"text":e.data_point.text,"start_offset":e.data_point.start_position,"end_offset":e.data_point.end_position})
        with open("prediction.jsonl",'a') as out:
            json.dump(json.dumps(outp),out)
            out.write('\n')

In [ ]:
numobs = 5

print("\033[1m Predictions \033[0m")
i = 0

for line in open("prediction.jsonl",'r'):
    print(json.loads(line))
    print()
    i += 1
    if i == numobs:
        break

print('\033[1m Ground Truth \033[0m')
i = 0

for line in open(folder_to_check + "train.jsonl"):
    #print(line)
    i += 1
    if i == numobs:
        break

 Predictions 
{"tweet_id": "1032071697221005312", "location_mentions": [{"text": "Kerala", "start_offset": 58, "end_offset": 64}, {"text": "Kerala", "start_offset": 204, "end_offset": 210}, {"text": "Karnataka", "start_offset": 223, "end_offset": 232}]}

{"tweet_id": "1034334390195822592", "location_mentions": []}

{"tweet_id": "1030975049510137856", "location_mentions": [{"text": "Kerala", "start_offset": 44, "end_offset": 50}]}

{"tweet_id": "1034481352341692417", "location_mentions": [{"text": "Kerala", "start_offset": 107, "end_offset": 113}, {"text": "India", "start_offset": 259, "end_offset": 264}]}

{"tweet_id": "1031529205673078784", "location_mentions": [{"text": "Kerala", "start_offset": 0, "end_offset": 6}, {"text": "india", "start_offset": 93, "end_offset": 98}, {"text": "kerala", "start_offset": 100, "end_offset": 106}, {"text": "Kerala", "start_offset": 208, "end_offset": 214}]}

 Ground Truth 


In [ ]:
import pickle

pickle.dump(model,open('model-loc.pkl' , 'wb'))